Klimov phase.

In [2]:
%display latex

We are going to compute a solution to $$c_{\kappa+\alpha, \mu} c_{\kappa,\mu}^* = c_{\alpha,\mu} \chi(\mu \alpha \kappa)$$ for the case of qubits. Klimov fix a basis in the field $GF(2^N)$, say, $\{\sigma_j\}$, such that
$$
\alpha = \sum_j \alpha_j \sigma_j, \quad a_j \in \mathbb Z_2.
$$

From the recurrence relation and substituting $\alpha = \kappa$ we obtain:
$$
c_{0, \mu} c_{\kappa,\mu}^* = c_{\kappa,\mu} \chi(\mu \kappa^2)
\implies
c_{\kappa,\mu}^2 = \chi(\mu \kappa^2)
\implies
c_{\kappa,\mu} = \sqrt{\chi(\mu \kappa^2)}
$$
We can solve this last equation for the $N$ basis elements $\kappa = \sigma_j$. Notice that we have two solutions for each $j$ because of the square root. This means that we have $2^N$ different solutions _sets_ for case $\alpha = \kappa$. Once we fix the signs of $c_{\kappa,\mu}$ for the basis elements, the rest of the coefficients $c_{\kappa,\mu}$ can be found by decomposing any element $\kappa$ in a linear combination of the $\sigma_j$. We have:
$$
\begin{align*}
c_{\alpha,\mu} = c_{\sum_{j=1} a_j \sigma_j,\mu}
&= c_{a_1 \sigma_1,\mu}c_{\sum_{j=2} a_j \sigma_j,\mu}\chi\left(\mu a_1 \sigma_1 \sum_{j=2} a_j \sigma_j\right) \\
&= \chi\left(\mu \sum_{k=1}^{N-1} a_k \sigma_k \sum_{j=k+1}^N a_j \sigma_j\right) \Pi_{l=1}^N c_{a_l \sigma_l, \mu}.
\end{align*}
$$

So the steps are quite simple. Choose a basis, select a sign for each of the $c_{\sigma_j,\mu}$ and then simply use this nasty formula to obtain the general solution. Let's verify the procedure with the example $GF(2^2)$.

In [3]:
N = 2
F = GF(2^N, 'x')
x = F.gen()

In [4]:
basis = [x, x^2]

In [11]:
chi = lambda k: exp(pi * I * int(k.trace()))

In [15]:
y = var('y')

In [18]:
mu = x^3
[solve([chi(mu * k^2) == y^2], y) for k in basis]

[[y == -I, y == I], [y == -I, y == I]]

We can see we have two choices for $\kappa = x$, and two choices for $\kappa = x^2$, therefore we obtain four possible solution sets for the given $\mu$. The solution for $\kappa = 0$ is fixed to be $1$ and for the third element of the field $x^3$, we simply apply the ugly formula which we now verify:

In [24]:
components = lambda k: [(k * el).trace() for el in basis]

In [31]:
basis_sols = [-I, I]
coefs = components(x^3)

s1 = 0
for k in range(N-1):
    s2 = sum([coefs[j] * basis[j] for j in range(k+1,N)])
    s1 += coefs[k] * basis[k] * s2
p1 = 1
for sol in basis_sols:
    p1 *= sol
chi(s1) * p1

1

Just as in the case of the Carlos Muñoz's solution, we will try to fit the recurrence relation solutions for _curves_ more general than those of straight lines. Klimov states that the regular curves, that is curves of the form $\beta = f(\alpha)$ can always be transformed to the horizontal straight line. This is done by a symplectic operators $P_f$ such that
$$P_f Z_\alpha P_f^* \sim Z_\alpha X_{f(\alpha)}.$$
This operator is unitary and can be defined as:
$$P_f = \sum_\kappa c_{\kappa,f} |\tilde \kappa\rangle \langle\tilde \kappa|,$$
where the coefficients $c_{\kappa,f}$ satisfy the recurrence relation:
$$c_{\kappa,f} c_{\kappa',f} = \chi(\kappa' f(\kappa)) c_{\kappa+\kappa',f}, \quad c_{0,f} = 1.$$

This equation admits several solutions for the qubit case. I tried using Carlos' solution but it wouldn't work. Let us now try Klimov's solution.

Suppose $\kappa = \kappa'$. Then
$$c_{\kappa,f}^2 = \chi(\kappa f(\kappa)).$$
Let's also assume that $f$ is biyective for the time being. Let us solve this equation for all of the basis elements. The curve in question is given by
$$f_\mu(\alpha) = \mu \alpha + \alpha^2 + \alpha^4.$$
We are now working with three qubits and so require the Galois field $GF(2^3)$.

In [242]:
F = GF(2^3, 'x')
x = F.gen()

y = var('y')

In [243]:
basis = [x^3, x^5, x^6]

def f(mu):
    return lambda k: mu * k + k**2 + k**4

In [244]:
mu = x
sols = []

for k in basis:
    sol = solve(chi(k * f(mu)(k)) == y^2, y, solution_dict=True)
    sols.append(sol[1]) # positive solutions only (choice)

sols

[{y: I}, {y: 1}, {y: I}]

Recall the formula used to compute the coefficients for arbitrary elements in the case of straight lines, that is for the map $f(\alpha) = \mu \alpha$:
$$
c_{\alpha,\mu} = c_{\sum_{j=1} a_j \sigma_j,\mu}
= \chi\left(\mu \sum_{k=1}^{N-1} a_k \sigma_k \sum_{j=k+1}^N a_j \sigma_j\right) \Pi_{l=1}^N c_{a_l \sigma_l, \mu}.
$$

Let's re-derive the general formula for $c_{\kappa,f}$. Recall the recurrence relation:
$$c_{\kappa+\kappa',f} = c_{\kappa,f} c_{\kappa',f} \chi(\kappa' f(\kappa)).$$
Let $\alpha = \sum_j a_j \sigma_j$, then:
$$
\begin{align*}
c_{\alpha,f}
&= c_{\sum_{j=1} a_j \sigma_j, f} \\
&= c_{a_1 \sigma_1, f} c_{\sum_{j=2} a_j \sigma_j, f} \chi\left[\left(\sum_{j=2} a_j \sigma_j\right) f(a_1 \sigma_1)\right] \\
&= c_{a_1 \sigma_1, f} \left\{
c_{a_2 \sigma_2, f} c_{\sum_{j=3} a_j \sigma_j, f} \chi\left[\left(\sum_{j=3} a_j \sigma_j\right)f(a_2 \sigma_2)\right]
\right\} \chi\left[\left(\sum_{j=2} a_j \sigma_j\right) f(a_1 \sigma_1)\right] \\
&= c_{a_1 \sigma_1, f}
c_{a_2 \sigma_2, f} c_{\sum_{j=3} a_j \sigma_j, f} 
\chi\left[\left(\sum_{j=3} a_j \sigma_j\right)f(a_2 \sigma_2) + \left(\sum_{j=2} a_j \sigma_j\right) f(a_1 \sigma_1)\right] \\
&= c_{a_1 \sigma_1, f}
c_{a_2 \sigma_2, f} \left\{
c_{a_3 \sigma_3, f} c_{\sum_{j=4} a_j \sigma_j, f} 
\chi\left[\left(\sum_{j=4} a_j \sigma_j\right) f(a_3 \sigma_3)\right]
\right\}
\chi\left[\left(\sum_{j=3} a_j \sigma_j\right)f(a_2 \sigma_2) + \left(\sum_{j=2} a_j \sigma_j\right) f(a_1 \sigma_1)\right] \\
&= c_{a_1 \sigma_1, f}
c_{a_2 \sigma_2, f} 
c_{a_3 \sigma_3, f} c_{\sum_{j=4} a_j \sigma_j, f} 
\chi\left[\left(\sum_{j=4} a_j \sigma_j\right) f(a_3 \sigma_3) + \left(\sum_{j=3} a_j \sigma_j\right)f(a_2 \sigma_2) + \left(\sum_{j=2} a_j \sigma_j\right) f(a_1 \sigma_1)\right] \\
&= \prod_{j=1}^N c_{a_j \sigma_j, f} \chi\left[
a_N \sigma_N f(a_{N-1} \sigma_{N-1}) + \left(\sum_{j=N-1}^N a_j \sigma_j\right) f(a_{N-2} \sigma_{N-2}) + \dots + \left(\sum_{j=2} a_j \sigma_j\right) f(a_1 \sigma_1)
\right] \\
&= \prod_{l=1}^N c_{a_l \sigma_l, f} \chi\left[
\sum_{k=1}^{N-1} \left(\sum_{j=k+1}^N a_j \sigma_j\right) f(a_k \sigma_k)
\right],
\quad a_d \in \mathbb Z_2, \forall d = 1,...,N, \quad c_{0,f} = 1.
\end{align*}
$$

It would probably be cleaner if I did this recursively.

In [245]:
def basis_c(a, l, sols):
    if a == 0:
        return 1
    return sols[l][y]

In [246]:
def solve_basis(basis, curve):
    sols = []
    for k in basis:
        sol = solve(chi(k * curve(k)) == y^2, y, solution_dict=True)
        sols.append(sol[1]) # positive solutions only (choice)
    return sols

In [247]:
def c(alpha, curve, sols=None):
    if not sols:
        sols = solve_basis(basis, curve)
    
    # Expand alpha in the basis
    comps = components(alpha)

    # Apply the formula
    s1 = 0
    for k in range(N-1):
        s2 = 0
        for j in range(k+1, N):
            s2 += comps[j] * basis[j]
        s1 += s2 * curve(comps[k] * basis[k])
        
    return prod([basis_c(a, l, sols) for l, a in enumerate(comps)]) * chi(s1)

Let's verify the recurrence relation:

In [248]:
for mu in F:
    curve = f(mu)
    sols = solve_basis(basis, curve)
    for k in F:
        for kp in F:
            lhs = c(k, curve, sols) * c(kp, curve, sols)
            rhs = chi(kp * curve(k)) * c(k + kp, curve, sols)
            if lhs != rhs:
                raise Exception('Recurrence relation does not hold!', mu)

The recurrence relation holds for $f_\mu$! So according to the paper, the $P_f$ operators should transform the standard basis into the curved basis. Before testing them out, we should be able to use this same method to generate the coefficients for $f_\mu(\alpha) = \mu \alpha$.

In [249]:
# for f(\alpha) = \mu \alpha
for mu in F:
    curve = lambda t: mu * t
    sols = solve_basis(basis, curve)
    for k in F:
        for kp in F:
            lhs = c(k, curve, sols) * c(kp, curve, sols)
            rhs = chi(kp * curve(k)) * c(k + kp, curve, sols)
            if lhs != rhs:
                raise Exception('Recurrence relation does not hold!', mu)

Works as well. But I have now realized that this probably won't work, and basically for the same reason as it doesn't work in the Galois ring construction. This solution requires an explicit solution for the basis and the given curve. That is to say, even when $\mu \xi = f_\mu(\xi)$, the actual calculation of the coefficients won't be the same as they depend on the curves themselves.

For the time being let's calculate the rotation operators for both sets of curves. Recall that:
$$
P_f = \sum_\kappa c_{\kappa,f} |\tilde \kappa\rangle \langle \tilde\kappa|.
$$

In [156]:
import numpy as np

In [191]:
def Proj(u, v=None):
    if not v:
        v = u
    return np.outer(u, v.conj().T)

Id = np.eye(2**N)

def Fourier():
    s = np.zeros((2**N, 2**N), dtype='complex128')
    for i, a in enumerate(F):
        for j, b in enumerate(F):
            s[i,j] = chi(a * b) / np.sqrt(2**N)
    return s
FF = Fourier()

def Z(a):
    return np.diag([chi(a * k) for k in F])

def X(b):
    return FF.conj().T * Z(b) * FF

# def D(a, b, p=1):
#     return phi(a, b, p) * Z(a) * X(b)

def V(xi, curve=None):
    if not curve:
        curve = lambda t: xi * t
    sols = solve_basis(basis, curve)
    
    s = np.zeros((2**N, 2**N), dtype='complex128')
    for i, k in enumerate(F):
        s += complex(c(k, curve, sols)) * Proj(FF[:,i])
    return s

In [250]:
mubs306 = [FF] + [V(mu) for mu in F]

In [203]:
from utils import checkMUBs

In [204]:
checkMUBs(mubs306)

MUBs are good!


In [251]:
mubs162 = [FF] + [V(mu, f(mu)) for mu in F]

In [206]:
checkMUBs(mubs162)

MUBs are good!


So now we have to verify that Sainz's formula does not work. The equations we want to verify are equations (40) to (42). Equation (40) gives us
$$
|\langle \psi_{\mu, \beta - \mu \alpha} | \psi_\kappa^{f_\nu} \rangle|^2
= \frac{1}{d} \sum_\xi c_{\xi, f_\nu(\xi)} c_{\xi, \mu\xi}^* \chi(\xi(-\kappa+\beta-\mu\alpha))
\delta_{\mu\xi, f_\nu(\xi)}.
$$

In [252]:
np.abs(mubs306[1][:,0].conj().T @ mubs162[1][:,5])**2

0.24999999999999983

In [253]:
def toInt(k):
    return list(F).index(k)

In [256]:
# Verification of the equation (40).

for s in F: 
    sols_curve = solve_basis(basis, f(s)) # basis solutions for f
    
    for mu in F:
        sols_line = solve_basis(basis, lambda t: mu * t) # basis sols for lines
        
        for alpha in F:
            for beta in F:
                for k in F:    
                    nu = beta - mu * alpha
    
                    psi_mu_nu = mubs306[toInt(mu)+1][:,toInt(nu)]
                    psi_f_k = mubs162[toInt(s)+1][:,toInt(k)]
    
                    inp = abs(psi_mu_nu.conj().T @ psi_f_k)**2
    
                    ss = 0
                    for xi in F:
                        c_xif = complex(c(xi, f(s), sols_curve))
                        c_ximu = complex(c(xi, lambda t: mu * t, sols_line))
                        char = complex(chi(xi * (k + nu)))
                        d = int(mu * xi == f(s)(xi))
                        ss += c_xif * conjugate(c_ximu) * char * d / (2**N)

                    if not np.isclose(inp, ss):
                        print(np.round(inp, 3), np.round(ss, 3))
                        raise Exception('No equality!', mu, nu, s, k)
print('Equation (40) is valid for all (a,b) and all curves!')

Equation (40) is valid for all (a,b) and all curves!


As expected, equation (40) is valid. And now for the verification of (42):
$$
\sum_\mu |\langle \psi_{\mu, \beta - \mu \alpha} | \psi_\kappa^{f_\nu} \rangle|^2
= 1 + \frac{1}{d} \sum_{\xi \in F^*} \chi(\xi(-\kappa + \beta - f_\nu(\alpha))),
$$

In [270]:
# Equation (42)

alpha = x
beta  = x
for s in F:
    sols_curve = solve_basis(basis, f(s)) # basis solutions for f
    for k in F:
        inps = 0.0
        for mu in F:
            nu = beta - mu * alpha
            # Inner product sum
            psi_mu_nu = mubs306[toInt(mu)+1][:,toInt(nu)]
            psi_f_k = mubs162[toInt(s)+1][:,toInt(k)]
            inps += np.abs(psi_mu_nu.conj() @ psi_f_k)**2

        # Right hand side of (42)
        s42 = 1.0
        for xi in list(F)[1:]:
            s42 += complex(chi(xi * (k + beta + f(s)(alpha)))) / (2**N)
    
        if not np.isclose(inps, s42):
            print(np.round(inps, 4), np.round(s42, 4), s, k)
            raise Exception('Sums do not match!')

1.625 (1.875+0j) 0 0


Exception: Sums do not match!

Let's write down this particular counter example. First recall that the right hand side of (40) is 
$$\sum_\mu \left(\frac{1}{d} \sum_\xi c_{\xi, f_\nu(\xi)} c_{\xi, \mu\xi}^* \chi(\xi(-\kappa+\beta-\mu\alpha))
\delta_{\mu\xi, f_\nu(\xi)}\right),$$
and the right hand side of (42) is given by:
$$1 + \frac{1}{d} \sum_{\xi \in F^*} \chi(\xi(-\kappa + \beta - f_\nu(\alpha)))$$

The faulty step in the proof is summing the first equation over all possible values of $\mu$. Using the delta function, the only non-zero terms of the sum are when $\mu \xi = f_\nu(\xi)$, therefore one naively thinks that
$$
c_{\xi, f_\nu(\xi)} = c_{\xi, \mu\xi}.
$$
But this is not the case, since the actual _calculation_ of the coefficient depends on the curves, and not only the image of $\xi$. Let $x$ be the generator of the field $GF(2^3)$ and consider the point $(\alpha,\beta) = (x,x)$, along with the curve parameters $\nu = 0$ and $\kappa = 0$. We are dealing with the vectors $|\psi_{\mu,\beta-\mu\alpha}\rangle = |\psi_{0,x}\rangle$ and $|\psi_{\kappa}^{f_\nu}\rangle = |\psi_0^{f_0}\rangle$.

In [280]:
alpha = x
beta = x
nu = 0
k = 0

sols_curve = solve_basis(basis, f(nu))
for mu in F:
    line = lambda t: mu * t
    sols_line = solve_basis(basis, line)
    for xi in F:
        if mu * xi == f(nu)(xi):
            cmu = c(xi, line, sols_line)
            cf  = c(xi, f(nu), sols_curve)
            if cmu != cf:
                print(mu, '\t', xi, '\t', cmu, '\t', cf)

0 	 1 	 1 	 -1


Notice that for $\mu = 0$ and $\xi = 1$, we have $\mu \xi = 0$ and $f_0(\xi) = \xi^2 + \xi^4 = 0$, and so $\delta_{\mu\xi, f_0(\xi)} = 1$. But for the rotation coefficients we obtain:

$$
\begin{align*}
c_{1,0}
&= \chi\left(0 \cdot\sum_{k=1}^{N-1} a_k \sigma_k \sum_{j=k+1}^N a_j \sigma_j\right) \prod_{l=1}^N c_{a_l \sigma_l, 0} \\
&= \prod_{l=1}^N c_{a_l \sigma_l, 0}, \quad a_l = Tr(1 \cdot \sigma_l).
\end{align*}
$$

In [281]:
components(F(1))

[1, 1, 1]

En la base elegida $\xi = x^3 + x^5 + x^6$, es decir, $a_l = 1$ para todo $l$.

In [282]:
solve_basis(basis, lambda t: F(0) * t)

[{y: 1}, {y: 1}, {y: 1}]

Las soluciones para la base y la recta parametrizada por $\mu = 0$ es, (elegiendo las raíces positivas), $c_{x^3,0} = c_{x^5,0} = c_{x^6,0} = 1$. Por lo tanto
$$
\prod_{l=1}^3 c_{a_l \sigma_l,0}
= c_{\sigma_1,0} \cdot c_{\sigma_2,0} \cdot c_{\sigma_3,0}
= c_{x^3,0} \cdot c_{x^5,0} \cdot c_{x^6,0}
= 1.
$$

We now calculate $c_{1,f_0}$ choosing the same signs for the basis solutions.
$$
\begin{align*}
c_{1,f_0}
&= \prod_{l=1}^N c_{a_l \sigma_l, f} \chi\left[
\sum_{k=1}^{N-1} \left(\sum_{j=k+1}^N a_j \sigma_j\right) f_0(a_k \sigma_k)
\right], \quad c_{0,f} = 1.
\end{align*}
$$

In [287]:
solve_basis(basis, f(F(0)))

[{y: 1}, {y: 1}, {y: 1}]

Therefore $\prod_{l=1}^N c_{a_l\sigma_l,f} = 1$. On the other hand:

In [291]:
[f(F(0))(k) for k in basis]

[x, x^2 + x, x^2]

And the argument of $\chi$ can be easily calculated:
$$
\begin{align*}
\sum_{k=1}^{2} \left(\sum_{j=k+1}^3 a_j \sigma_j\right) f_0(a_k \sigma_k)
&= \left(\sum_{j=2}^3 a_j \sigma_j\right) f_0(a_1 \sigma_1) + 
\left(\sum_{j=3}^3 a_3 \sigma_3\right) f_0(a_2 \sigma_2) \\
&= \left(a_2 \sigma_2 + a_3 \sigma_3\right) f_0(a_1 \sigma_1) + 
\left(a_3 \sigma_3\right) f_0(a_2 \sigma_2) \\
&= (1 \cdot x^5 + 1 \cdot x^6) x + (1 \cdot x^6)(x^2+x) \\
&= x^6 + x^7 + x^8 + x^7 \\
&= x^6 + x^8 \\
&= x^2 + x + 1 \\
&= x^5.
\end{align*}
$$

In [297]:
chi(x^5)

-1

But $\chi(x^5) = -1$, so $c_{1,f_0} = (1)(-1) = -1 \neq 1 = c_{1,0}$!

---

Now let's try something different. In the 2017 paper, Klimov states that the MUBs with factorization $(1,6,2)$ can be obtained _from_ the standard one by means of a transformation $P_f$, with the curve $f(\alpha) = \alpha + \alpha^2 + \alpha^4$. This corresponds to our curve $f_1$. Before trying anything else, let's see if we can obtain valid MUBs.

In [299]:
Pf = V(F(1), f(F(1)))

In [309]:
mubs162_2 = [mubs306[0]] + [Pf @ mubs306[j] for j in range(1, 2**N+1)]

In [311]:
checkMUBs(mubs162_2)

MUBs are good!


In [320]:
def Wootters(a, b):
    op = Proj(FF[:, toInt(a)])
    for xi in F:
        for nu in F:
            d = int(b == xi * a + nu)
            v = mubs306[toInt(xi)+1][:, toInt(nu)]
            op += d * Proj(v)
    return op - Id

def Wigner(state, a, b):
    return (state @ Wootters(a, b)).trace()

def WignerMatrix(state):    
    W = np.zeros((2**N, 2**N), dtype='float64')
    for i, a in enumerate(F):
        for j, b in enumerate(F):
            W[i, j] = real(
                Wigner(state, a, b)
            ) / 2**N
    return W

def WW(state, rnd=True):
    return np.round(WignerMatrix(state), 3)

In [336]:
WW(Proj(mubs162_2[5][:,5]))

array([[ 0.031, -0.031,  0.031,  0.031,  0.094,  0.031, -0.031, -0.031],
       [ 0.031, -0.031,  0.031,  0.031, -0.031,  0.031, -0.031,  0.094],
       [-0.031,  0.031, -0.031,  0.094,  0.031, -0.031,  0.031,  0.031],
       [ 0.031, -0.031,  0.031,  0.031,  0.094,  0.031, -0.031, -0.031],
       [-0.031,  0.031,  0.094, -0.031,  0.031, -0.031,  0.031,  0.031],
       [-0.031,  0.031, -0.031,  0.094,  0.031, -0.031,  0.031,  0.031],
       [-0.031,  0.031,  0.094, -0.031,  0.031, -0.031,  0.031,  0.031],
       [ 0.031, -0.031,  0.031,  0.031, -0.031,  0.031, -0.031,  0.094]])

In [348]:
np.allclose(Pf @ Pf.conj().T, Id)

True

In [349]:
np.sum(WW(Proj(mubs162_2[5][:,5])))

1.0